In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType ,LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType
import datetime

common_event = StructType().add("trade_dt",DateType(),True)\
                    .add("rec_type", StringType(), True)\
                    .add("symbol", StringType(), True)\
                    .add("exchange", StringType(), True)\
                    .add("event_tm", TimestampType(),True)\
                    .add("event_seq_nb", IntegerType(), True)\
                    .add("arrival_tm", TimestampType(), True)\
                    .add("trade_pr", DecimalType(), True)\
                    .add("bid_pr", DecimalType(), True)\
                    .add("bid_size", IntegerType(), True)\
                    .add("ask_pr", DecimalType(), True)\
                    .add("ask_size", IntegerType(), True)\
                    .add("partition", StringType(), True)

In [0]:
trade_common = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/data/data.parquet/partition=T")

In [0]:
import pyspark.sql.functions as F
trade = trade_common.select('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'arrival_tm', 'trade_pr')

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
trade_latest = trade.withColumn("rank", row_number().over(Window.partitionBy(trade.trade_dt, trade.symbol, trade.exchange, trade.event_tm, trade.event_seq_nb).orderBy(trade.arrival_tm.desc()))).filter(col("rank")==1).drop("rank")

In [0]:
trade.write.mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet", partitionBy = "trade_dt")

In [0]:
trade_common = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/partition=T")

In [0]:
import pyspark.sql.functions as F
trade = trade_common.select('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'file_tm', 'bid_pr')

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
trade_latest = trade.withColumn("rank", row_number().over(Window.partitionBy(trade.trade_dt, trade.symbol, trade.exchange, trade.event_tm, trade.event_seq_nb).orderBy(trade.file_tm.desc()))).filter(col("rank")==1).drop("rank")

In [0]:
trade.write.mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet", partitionBy = "trade_dt")

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/_committed_6832646475687657258,_committed_6832646475687657258,35
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/partition=B/,partition=B/,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/partition=Q/,partition=Q/,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/partition=T/,partition=T/,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/trade_dt=2020-08-05/,trade_dt=2020-08-05/,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/trade_dt=2020-08-06/,trade_dt=2020-08-06/,0


In [0]:
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/_committed_8353887648582217260,_committed_8353887648582217260,35
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/partition=Q/,partition=Q/,0
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/partition=T/,partition=T/,0
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/trade_dt=2020-08-05/,trade_dt=2020-08-05/,0
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/trade_dt=2020-08-06/,trade_dt=2020-08-06/,0
